In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/github/ids

/content/drive/MyDrive/github/ids


In [6]:
!git a

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	colab.ipynb

nothing added to commit but untracked files present (use "git add" to track)
